# Using GABase

A Python class called `GABase` is defined in [ga.py](ga.py) and it can be used to derive new classes for solving specific problems. `GABase` is an abstract class that contains the following 5 abstract methods:

```python
obj(self, s)
initialization(self)
obj2fitness(self)
crossover(self, s1, s2)
mutation(self, s)
```

These abstract methods must be implemented in the derived classes. In addition, the derived class may also override other methods to accommodate problem-specific requirements. Two examples of overriding `GABase` are provided in the repository for solving a numerical optimization problem using binary strings ([ga_base_binary.py](ga_base_binary.py)) and the p-median problem ([ga_base_pmed.py](ga_base_pmed.py)). The details of these two examples are shown below.

The `ga` module also contains another class called `Parameters` that can be used to store the parameters used by a GA. This class includes the following commonly used parameters for a GA:

```
popsize      - population size, integer
numgen       - number of generations, integer
pcrossover   - probability of crossover, float between 0 and 1
pmutation    - probability of mutation, float between 0 and 1
elitism      - use of elitism, boolean
```


## Numerical optimization using binary strings

We write a new GA class to solve the following problem:

$$\max x_1^2 - x_2 - x_3 + 1$$

where $x_1$, $x_2$, and $x_3$ are integers in the range of 0 and 7. This is a simple problem where the optimal solution is $x_1 = 7, x_2 = 0, x_3 = 0$. We will a binary string to encode a solution to the problem.

The following code is from [ga_base_binary.py](ga_base_binary.py).

We first import the `ga` module and necessary other modules (`random` in this case):

In [1]:
from ga import *
from random import randint

Then we define our own class using `GABase` as the parent class. Since we don't have an additional parameters, we simply call the `__init__` method from the super class. However, we are going to use a binary representation here and therefore we need to override the `decode` method. Then we must override the five abstract methods so that we know how to solve the specific problem. A typical single-point crossover is implemented here.

In [6]:
class GABin(GABase):
    def __init__(self, params):
        super().__init__(params)

    def decode(self, s):
        def _decode(s):
            num = 0
            n = len(s)
            for i in range(n):
                c = s[n-i-1]
                num += int(c) * 2 ** i
            return num
        return _decode(s[:3]), _decode(s[3:6]), _decode(s[6:])

    def obj(self, x):
        return x[0] ** 2 - x[1] - x[2] + 1

    def obj2fitness(self):
        low = self.worst()[0]
        minimal = (self.best()[0] - low) * 0.1
        fitnesses = [val - low + minimal for val in self.objs]
        return fitnesses

    def initialization(self):
        self.population = []
        for i in range(self.params.popsize):
            sol = ''.join(['1' if flip(0.5) else '0' for i in range(9)])
            self.population.append(sol)
        self.evaluation()

    def crossover(self, s1, s2):
        if not flip(self.params.pcrossover):
            return [s1, s2]
        n = len(s1)
        x = randint(0, n-1)
        s1x = s1[:x] + s2[x:]
        s2x = s2[:x] + s1[x:]
        return [s1x, s2x]

    def mutation(self, s):
        mutated = ''
        mutate = lambda i: '1' if i=='0' else '0'
        for c in s:
            if flip(self.params.pmutation):
                mutated += mutate(c)
            else:
                mutated += c
        return mutated


The new GA class is ready to use. We need to create a `Parameter` object and then create a `GA` object.

In [8]:
params = Parameters(
    popsize = 10,
    numgen = 10,
    pcrossover = 0.9,
    pmutation = 0.1,
    elitism = True)
    
myga = GABin(params)
res = myga.run()
print(res)

print([myga.run()[0] for _ in range(10)])


(49, 33.7)
[50, 49, 50, 50, 50, 50, 49, 48, 50, 50]


## Solving the p-median problem

Now we demonstrate how to implement a GA for the p-median problem where different operations must be developed and addition parameters are needed.

The following code example is from [ga_base_pmed.py](ga_base_pmed.py).

In [5]:
from ga import *
from random import randint, sample

The following is a new class that is derived from the parent class of `GABase`. Here, the `__init__` method includes additional code to make sure the data needed for the p-median is passed to the class. More specific, we use a dictionary (`owndata`) to store each of the required data items (`n`, `distmatrix`, and `p`). More of this dictionary can be found below.

We then override the abstract methods using the code described in another tutorial for [GA introduction](genetic%20algorithms.ipynb).

In [9]:
class GAPMed(GABase):
    def __init__(self, params, owndata):
        super().__init__(params)
        self.n = owndata['n']
        self.distmatrix = owndata['distmatrix']
        self.p = owndata['p']

    def obj(self, s):
        n = len(self.distmatrix)
        d_total = 0
        for i in range(self.n):
            d_min = float('inf')
            for j in s:
                if self.distmatrix[i][j] < d_min:
                    d_min = self.distmatrix[i][j]
            d_total += d_min
        return d_total

    def obj2fitness(self):
        return [1/val for val in self.objs]

    def initialization(self):
        self.population = []
        for i in range(self.params.popsize):
            sol = sample(range(self.n), self.p)
            self.population.append(sol)
        self.evaluation()

    def crossover(self, s1, s2):
        if not flip(self.params.pcrossover):
            return [s1]
        child = list(set(s1 + s2))
        while len(child) > self.p:
            d = float('inf')
            to_remove = -1
            for i in child:
                c1 = [j for j in child if j != i]
                d1 = self.obj(c1)
                if d1 < d:
                    d = d1
                    to_remove = i
            child = [j for j in child if j != to_remove]
        return [child]

    def mutation(self, s):
        for i in range(len(s)):
            if not flip(self.params.pmutation):
                continue
            while True:
                j = randint(0, self.n-1)
                if not j in s:
                    s[i] = j
                    break
        return s


The following is the data required for the p-median problem.

In [10]:
owndata = {
    'n': 8,
    'distmatrix': [
        [0, 3, 13, 5, 12, 16, 17, 20],
        [3, 0, 10, 8, 9, 13, 14, 23],
        [13, 10, 0, 8, 9, 3, 14, 15],
        [5, 8, 8, 0, 17, 11, 22, 15],
        [12, 9, 9, 17, 0, 6, 5, 16],
        [16, 13, 3, 11, 6, 0, 11, 12],
        [17, 14, 14, 22, 5, 11, 0, 11],
        [20, 23, 15, 15, 16, 12, 11, 0]],
    'p': 2
}


And we need to specify the GA parameters:

In [11]:
params = Parameters(
    popsize = 4,
    numgen = 10,
    pcrossover = 0.9,
    pmutation = 0.1,
    elitism = False,
    minmax = 'min')


Now we can create a new object of the `GAPMed` class and run it. The optimal solution to this specific problem has a total distance of 40.

In [16]:
myga = GAPMed(params, owndata)
res = myga.run()
print(res[0])

40


We can further test the GA performance using various parameter settings. Here is a test on the mutation probability:

In [14]:
pmutate = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
paramx = [Parameters(4, 10, 0.9, pm, True, 'min') for pm in pmutate]

def multirun(prog, n, best):
    all_best = []
    for _ in range(n):
        res = prog.run()
        all_best.append(res[0])
    return sum([1 if res==best else 0 for res in all_best])

res = [multirun(myga, 100, 40) for myga in [GAPMed(par, owndata) for par in paramx]]

print('{:15} {}'.format('mutation', 'obj'))
for i in range(len(res)):
    print('{:15} {}'.format(pmutate[i], res[i]))


mutation        obj
              0 31
            0.1 69
            0.2 82
            0.3 89
            0.4 87
            0.5 91
            0.6 88
            0.7 87
            0.8 80
            0.9 81
              1 75
